In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [2]:
# Gather Data
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=['PRICE'])

In [3]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

#property_stats = np.ndarray(shape=(1,11))
#property_stats[0][CRIME_IDX]=features['CRIM'].mean()
#property_stats[0][ZN_IDX]=features['ZN'].mean()
#property_stats[0][CHAS_IDX]=features['CHAS'].mean()
#property_stats[0][RM_IDX]=0.02
#property_stats[0][PTRATIO_IDX]=0.02

property_stats = features.mean().values.reshape(1,11)

In [4]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)
mse = mean_squared_error(target, fitted_vals)
rmse = np.sqrt(mse)


In [5]:
def get_log_estimate(nr_rooms, 
                    students_per_classroom, 
                    next_to_river=False, 
                    high_confidence=True):
    
    # Configure property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0

    # Make prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    # Calc Range
    if high_confidence:
        upper_bound = log_estimate + 2*rmse
        lower_bound = log_estimate - 2*rmse
        interval = 95
    else:
        upper_bound = log_estimate + rmse
        lower_bound = log_estimate - rmse
        interval = 68
    
    return log_estimate, upper_bound, lower_bound, interval

In [6]:
get_log_estimate(3,20, next_to_river=True, high_confidence=False)

(2.7767581914803987, 2.964270326677529, 2.5892460562832684, 68)

In [7]:
np.median(boston_dataset.target)

21.2

In [8]:
ZILLOW_MEDIAN_PRICE = 583.3
SCALE = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

log_est, upper, lower, conf = get_log_estimate(9, students_per_classroom=15, 
                                               next_to_river=False, high_confidence=False)

doll_est = round(np.e**log_est * 1000 * SCALE, -3)
doll_upp = round(np.e**upper * 1000 * SCALE, -3)
doll_low = round(np.e**lower * 1000 * SCALE, -3)

print(f'The estimated property value is {doll_est}')
print(f'At the {conf}% confedence the valuation range is')
print(f'USD {doll_low} at the lower end to USD {doll_upp} at the high end.')

The estimated property value is 827000.0
At the 68% confedence the valuation range is
USD 685000.0 at the lower end to USD 997000.0 at the high end.


In [9]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    """Estimate the price of property in Boston
    
    Keyword arguments:
    rm -- number of rooms in the property
    ptratio -- number of students
    chas -- near river or not
    large_range --
    
    """
    
    
    if rm <1 or ptratio < 1:
        print('That is unreal. Try again')
        return
    
    log_est, upper, lower, conf = get_log_estimate(rm, students_per_classroom=ptratio, 
                                               next_to_river=chas, high_confidence=large_range)

    doll_est = round(np.e**log_est * 1000 * SCALE, -3)
    doll_upp = round(np.e**upper * 1000 * SCALE, -3)
    doll_low = round(np.e**lower * 1000 * SCALE, -3)

    print(f'The estimated property value is {doll_est}')
    print(f'At the {conf}% confedence the valuation range is')
    print(f'USD {doll_low} at the lower end to USD {doll_upp} at the high end.')
    

In [10]:
get_dollar_estimate(rm=6, ptratio=13, chas=True)

The estimated property value is 754000.0
At the 95% confedence the valuation range is
USD 518000.0 at the lower end to USD 1097000.0 at the high end.


In [11]:
import boston_valuation as val

In [13]:
val.get_dollar_estimate(7,12,True)

The estimated property value is 857000.0
At the 95% confedence the valuation range is
USD 589000.0 at the lower end to USD 1247000.0 at the high end.
